In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats.mstats import winsorize
from google.colab import drive, files
import io

# Cargar datos y dividir dataset

In [2]:
from tensorflow.random import set_seed
set_seed(234730)

In [3]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test_without_label.csv')

In [4]:
train['id'] = train['id'].astype(str)
train['tweet'] = train['tweet'].astype(str)
train['label'] = train['label'].astype(str)

In [5]:
train.drop(['id'], axis=1)
train.set_index("id", inplace = True)

In [6]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
# parseo de category
train[['label']] = train[['label']].apply(lambda col: label_encoder.fit_transform(col))

# Stopwords, lemmatize, punctuacion
Hay muchas palabras que estan repetidas como "august", "august,", "august.", "august:", etc. Para resolver esto vamos a sacar la punctuación antes de hacer el lemmatizer. Por ahora vamos a dejar algunos símbolos como los "#", quizas los hastags pueden tener información importante.

Como en el órden de ejecución del CountVectorizer primero se hace el lemmatizing,  no podemos ponerlo en las stop words. Por lo tanto lo vamos a hacer en el mismo método de lemmatizing.

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer
import nltk

nltk.download('wordnet')
Lemmatizer = WordNetLemmatizer()
#analyzer = CountVectorizer().build_analyzer()

from zipfile import ZipFile
file_loc = '/root/nltk_data/corpora/wordnet.zip'
with ZipFile(file_loc, 'r') as z:
  z.extractall('/root/nltk_data/corpora/')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [8]:
#All_punct = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
All_punct = '''!()-[]{};:'"\,<>./?%^&*_~'''
#CountVectorizer con stopwords de Natural Language Toolkit
nltk.download('stopwords')

def is_Not_Link(word):
  return word[0:4] != "http"

def remove_Punctuation(doc_split):
  for i in range(len(doc_split)):
    word = doc_split[i];
    if is_Not_Link(word):
      for elements in word:
        if elements in All_punct:
          doc_split[i] = word.replace(elements, "")
  return doc_split

def remove_Stopwords(doc_split):
  doc_with_Stopwords = doc_split.copy();
  for i in range(len(doc_split)):
    word = doc_split[i];
    if word in nltk.corpus.stopwords.words('english'):
      doc_with_Stopwords.remove(word);

  return doc_with_Stopwords

def new_analyzer(doc):
  doc_split = doc.split();
  doc_split = remove_Punctuation(doc_split);
  doc_split = remove_Stopwords(doc_split);
  return (Lemmatizer.lemmatize(w.lower()) for w in doc_split);


nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

# Ingenieria de atributos

## Largo del documento
Uno de los atributos que nos interesa investigar es el largo de los atributos

In [10]:
train['largoDocumento'] = train.tweet.str.len()

## Train / Validation
Al final de la ingeniería dividimoms en train y validation

In [11]:
from sklearn.model_selection import train_test_split

X_train, X_validation, Y_train, Y_validation = train_test_split(
    train.drop('label', axis=1),
    train['label'],
    test_size=(1.0/3), random_state=42)

## Fit

In [13]:
TF_IDF_Vectorizer = TfidfVectorizer(analyzer=new_analyzer, max_features=5000)
bag_of_words = TF_IDF_Vectorizer.fit(X_train["tweet"])

print(bag_of_words.get_feature_names())

['', '"im', '"its', '#', '#1', '#2020presidentialelection', '#21dayslockdown', '#acog2020', '#actnow', '#america', '#amitabhbachchan', '#asymptomatic', '#auspol', '#bacteria', '#beer', '#bihar', '#breakfast', '#cdnpoli', '#china', '#codecheck', '#colorado', '#connecticut', '#containmentzones', '#corona', '#coronabeer', '#coronacheck', '#coronaoutbreak', '#coronaupdate', '#coronaupdates', '#coronaupdatesindia', '#coronavirus', '#coronavirusfacts', '#coronavirusindia', '#coronavirusoutbreak', '#coronaviruspandemic', '#coronavirusupdate', '#coronavirusupdates', '#coronawarriors', '#coronawatch', '#coronil', '#covid', '#covid-19', '#covid19', '#covid19associated', '#covid19india', '#covid19like', '#covid19nigeria', '#covid19pakistan', '#covid19science', '#covid2019', '#covidindiaseva', '#covidupdates', '#covidview', '#covidー19', '#datoscoronavirus', '#death', '#delhi', '#disease', '#donaldtrump', '#dranthonyfauci', '#dyk', '#facemasks', '#factcheck', '#fakenews', '#faq', '#fatalityrate', '

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


## Transform

In [26]:
bag_of_words = TF_IDF_Vectorizer.transform(X_train["tweet"])
bag_of_words = pd.DataFrame(bag_of_words.toarray(), index = X_train.index, columns = TF_IDF_Vectorizer.get_feature_names())

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


## Añadir atributos al nuevo dataframe de TFIDF

In [ ]:
bag_of_words

In [ ]:
X_train

In [28]:
bag_of_words['largoDocumento'] = X_train['largoDocumento']

# Red Neuronal

In [30]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.layers import LeakyReLU

In [31]:
adam_optimizer='Adam'
loss='binary_crossentropy'
metric='accuracy'

model = Sequential()
model.add(Dense(20, input_shape=(bag_of_words.shape[1],)))
model.add(Activation(LeakyReLU(alpha=0.3)))
model.add(Dense(30, use_bias=True))
model.add(Activation(LeakyReLU(alpha=0.3)))
model.add(Dense(1, use_bias=True))
model.add(Activation('sigmoid'))
model.compile(loss=loss, optimizer=adam_optimizer, metrics=[metric])

In [ ]:
X_train_array = np.asarray(bag_of_words)
Y_train_array = np.asarray(Y_train)

training = model.fit(X_train_array, Y_train_array, epochs=200, batch_size=100, validation_split=0.2)

In [33]:
import tensorflow.keras as keras

model.save("basic_model")
saved_model = keras.models.load_model("basic_model")

In [34]:
X_Validation_Bag = TF_IDF_Vectorizer.transform(X_validation["tweet"])
X_Validation_Bag_df = pd.DataFrame(X_Validation_Bag.toarray(), index = X_validation.index,columns = TF_IDF_Vectorizer.get_feature_names())

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
X_Validation_Bag_df

In [36]:
X_Validation_Bag_df['largoDocumento'] = X_validation['largoDocumento']

In [38]:
X_Validation_Bag_df_array = np.asarray(X_Validation_Bag_df)
Y_Validation_Predict = saved_model.predict(X_Validation_Bag_df_array)

90/90 [==============================] - 0s 2ms/step


In [ ]:
Y_Validation_Predict

In [39]:
def transformPrediction(df, predNumber, predLabel):
  copy = df.copy()
  for i in range(len(copy)):
    if copy[i][0] > 0.5:
      predNumber.append([1])
      predLabel.append(["real"])
    else:
      predNumber.append([0])
      predLabel.append(["fake"])


In [40]:
predValidationNumbers = []
predValidationLabels = []
transformPrediction(Y_Validation_Predict, predValidationNumbers, predValidationLabels)

In [41]:
validationNumbers = []
validationLabels = []
copy = np.asarray(Y_validation).copy()
for i in range(len(copy)):
  if copy[i] > 0.5:
    validationNumbers.append([1])
    validationLabels.append(["real"])
  else:
    validationNumbers.append([0])
    validationLabels.append(["fake"])

In [ ]:
np.asarray(Y_validation)

array([1, 1, 1, ..., 0, 1, 0])

In [42]:
from sklearn.metrics import accuracy_score
print(accuracy_score(predValidationLabels,validationLabels))

0.9201121233356693


# K fold

In [53]:
from sklearn.metrics import accuracy_score

In [54]:
import math
k = 5

def k_folds(X, y, k):
  assert(len(X) == len(y))
  folds_X = []
  folds_y = []
  initial_pos = 0

  for i in range(k):
    to_pos = min(math.ceil(len(X)/k)*(i+1), len(X))
    
    x_fold = X[initial_pos:to_pos]
    y_fold = y[initial_pos:to_pos]

    folds_X.append(x_fold)
    folds_y.append(y_fold)
    initial_pos = to_pos

  return folds_X, folds_y

In [55]:
def evaluate(model, X_to_eval, Y_to_eval):
    Y_Prediction = model.predict(np.asarray(X_to_eval))
    numbers = []
    prediction_labels = []
    transformPrediction(Y_Prediction, numbers, prediction_labels)

    validationLabels = []
    copy = np.asarray(Y_to_eval).copy()
    for i in range(len(copy)):
      if copy[i] > 0.5:
        validationLabels.append(["real"])
      else:
        validationLabels.append(["fake"])

    return accuracy_score(prediction_labels, validationLabels)

In [56]:
def k_fold_cross_val(folds_X, folds_y, model):
  assert(len(folds_X) == len(folds_y))
  X_to_train = []
  y_to_train = []
  X_to_eval = 0
  y_to_eval = 0
  scores = []
  for i in range(len(folds_X)):
    X_to_train = []
    y_to_train = []
    for j in range(len(folds_X)):
      if  i == j:
        X_to_eval = folds_X[i]
        y_to_eval = folds_y[i]
      else:
        X_to_train.append(folds_X[j])
        y_to_train.append(folds_y[j])
    
    X_train = np.concatenate(X_to_train)
    y_train = np.concatenate(y_to_train)
    model.fit(X_train,y_train, epochs=200, batch_size=100, validation_split=0.2)
    score = evaluate(model, X_to_eval, y_to_eval)
    scores.append(score)
  return np.mean(np.array(scores)), scores

In [57]:
folds_X, folds_y = k_folds(np.asarray(bag_of_words), np.asarray(Y_train), 5)


In [ ]:
k_fold_result_mean, k_fold_result_arr = k_fold_cross_val(folds_X, folds_y, model)

In [59]:
k_fold_result_mean

0.9642196497988605

In [60]:
k_fold_result_arr

[0.9273204903677759,
 0.9737302977232924,
 0.9964973730297724,
 1.0,
 0.9235500878734623]

# Test

In [ ]:
test['id'].astype('str')
test['tweet'].astype('str')

test_bag = TF_IDF_Vectorizer.transform(test["tweet"])
test_bag = pd.DataFrame(test_bag.toarray(), columns = TF_IDF_Vectorizer.get_feature_names())

test_bag_array = np.asarray(test_bag)
test_prediction = saved_model.predict(test_bag_array)

copy = test_prediction.copy()
predNumber = [];
predLabel = [];
for i in range(len(copy)):
  if copy[i][0] > 0.5:
    predNumber.append([1])
    predLabel.append(["real"])
  else:
    predNumber.append([0])
    predLabel.append(["fake"])

prediction = pd.DataFrame(predLabel, columns=['label'])
prediction.index += 1
prediction = prediction.to_csv('prediction.csv')

In [ ]:
from google.colab import files

files.download('prediction.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>